** Análise de Sentimentos – reviews_Clothing_Shoes_and_Jewelry Amazon

In [155]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from pymongo import MongoClient
import pprint
import numpy as np

client = MongoClient()
print (client)
db = client.amazon
clothing = db.clothing
#db.collection_names(include_system_collections=False)
#pprint.pprint(clothing.find_one())
clothing.find_one()

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


{'_id': ObjectId('5a022e00fce323bee830197c'),
 'asin': '0000031887',
 'classification': 'Positive',
 'helpful': [0, 0],
 'overall': 5.0,
 'reviewText': 'What can I say... my daughters have it in orange, black, white and pink and I am thinking to buy for they the fuccia one. It is a very good way for exalt a dancer outfit: great colors, comfortable, looks great, easy to wear, durables and little girls love it. I think it is a great buy for costumer and play too.',
 'reviewTime': '01 4, 2013',
 'reviewerID': 'A1RLQXYNCMWRWN',
 'reviewerName': 'Carola',
 'summary': 'I have buy more than one',
 'unixReviewTime': 1357257600}

**Ler arquivo de dados e conta a quantidade de linhas**

In [156]:
dataset = clothing.find()

#import gzip

#def parse(path):
 # g = gzip.open(path, 'rb')
  #for l in g:
   # yield eval(l)

#def getDF(path):
  #i = 0
  #df = {}
  #for d in parse(path):
    #df[i] = d
    #i += 1
  #return pd.DataFrame.from_dict(df, orient='index')

#df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')
#df.count();

In [157]:
dataset.count()

#df.count();

Unnamed: 0                   8199
Created At                   8199
Text                         8199
Geo Coordinates.latitude      104
Geo Coordinates.longitude     104
User Location                5489
Username                     8199
User Screen Name             8199
Retweet Count                8199
Classificacao                8199
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

**Conta a quantidade de linhas de reviews neutros, positivos e negativos**

In [158]:
#dataset[dataset.Classificacao=='Neutro'].count()

#for record in dataset:
 #   print(record['overall']) 
  #  print(record['classification']=='Positive')
   # break;

#dataset[dataset.classification=='Neutral'].count()

clothing.count({"classification": "Neutral"})

#df[df.classification=='Neutral'].count()

30425

In [159]:
#dataset[dataset.Classificacao=='Positivo'].count()

clothing.count({"classification": "Positive"})

#df[df.classification=='Positive'].count()

221597

In [160]:
#dataset[dataset.Classificacao=='Negativo'].count()

clothing.count({"classification": "Negative"})

#df[df.classification=='Negative'].count()

26655

** Pre-Processamento **

In [161]:
def PreprocessamentoSemStopWords(instancia):
    #remove links dos reviews
    #remove stopwords
    instancia = re.sub(r"http\S+", "", instancia).lower().replace(',','').replace('.','').replace(';','').replace('-','')
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [162]:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras=[]
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))    

In [163]:
def Preprocessamento(instancia):
    #remove links, pontos, virgulas,ponto e virgulas dos reviews
    #coloca tudo em minusculo
    instancia = re.sub(r"http\S+", "", instancia).lower().replace(',','').replace('.','').replace(';','').replace('-','').replace(':','')
    return (instancia)

In [164]:
instancia = PreprocessamentoSemStopWords('What can I say... my daughters have it in orange, black, white and pink and I am thinking to buy for they the fuccia one. It is a very good way for exalt a dancer outfit: great colors, comfortable, looks great, easy to wear, durables and little girls love it. I think it is a great buy for costumer and play too.')
print(instancia)
instancia = Stemming(instancia)
print(instancia)

say daughters orange black white pink thinking buy fuccia one good way exalt dancer outfit: great colors comfortable looks great easy wear durables little girls love think great buy costumer play
say daught orang black whit pink thinking buy fucc one good way exalt danc outfit: great col comfortabl look great easy we durabl littl girl lov think great buy costum play


** Separando reviews e suas classes
**

In [165]:
reviews = []
for r in clothing.find({}, { "_id": 0, "reviewText": 1} ):
    reviewText = PreprocessamentoSemStopWords(r['reviewText'])
    reviewText = Stemming(instancia)
    reviews.append(reviewText)

#print(len(reviews))
    
#print(reviews[0])
#print(reviews[1])

classifications = []
for c in clothing.find({}, { "_id": 0, "classification": 1} ):
    classifications.append(c['classification'])

classifications = np.asarray(classifications)

print(classifications)

#print(len(classifications))
#print(classifications.values)

#for cl in classifications :
 #   print(cl)

#reviews = df['reviewText'].values
#classifications = df['classification'].values

<class 'numpy.ndarray'>
['Positive' 'Positive' 'Positive' ..., 'Positive' 'Positive' 'Positive']


**Gerando o modelo**

In [166]:
vectorizer = CountVectorizer(analyzer="word")
freq_reviews = vectorizer.fit_transform(reviews)
modelo = MultinomialNB()
modelo.fit(freq_reviews, classifications)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

** Testando o modelo com algumas instâncias simples**

In [167]:
# defina instâncias de teste dentro de uma lista

In [168]:
testes = ["My 3yr old loved this tutu skirt in pink! Was hoping to order more in different colors.She had hardly used this,the stitching came apart in 2weeks.now it's lying in her closet..Altogether she wore it like 4-5 times for 20 mins or so.wish the stitching was of better quality to hold up while little ones wear it. Can't recommend.",
         "What can I say... my daughters have it in orange, black, white and pink and I am thinking to buy for they the fuccia one. It is a very good way for exalt a dancer outfit: great colors, comfortable, looks great, easy to wear, durables and little girls love it. I think it is a great buy for costumer and play too.",
        "Never GOT this item - but gave a 1 STAR because the replies from the SUPPLIER was GREAT.They tried to send the item more than once.My $ was refunded in a timely manner too.It was a shame I never got it for my daughter - it would of looked great with her OUTFIT for Dr. Seuss WEEK at school.Most original.Maybe next time.", 
       "I already own this particular Shining Image jewelry box in brown, so this was my second buy. It arrived with some of the leather scratched, even though it looked like I was the first one opening the box. This particular color pink also looked pretty bad in person. The quality of this box seemed lesser than the brown one I own.I returned this pink case for a refund without a problem.  Got another brown Shining Image jewelry case and it's fine!"]

In [169]:
freq_testes = vectorizer.transform(testes)

In [170]:
# Fazendo a classificação com o modelo treinado.

In [171]:
modelo.predict(freq_testes)

array(['Positive', 'Positive', 'Positive', 'Positive'],
      dtype='<U8')

** Avaliando o modelo **

In [172]:
# Fazendo o cross validation do modelo

In [173]:
resultados = cross_val_predict(modelo, freq_reviews, classifications, cv=10)
#len(resultados)

#print(classifications)
#len(classifications)

In [174]:
# Medindo a acurácia média do modelo

In [175]:
metrics.accuracy_score(classifications,resultados)

0.79517505929804033

In [176]:
# Medidas de validação do modelo

In [178]:
sentimento=['Positive','Negative','Neutral']
print (metrics.classification_report(classifications,resultados,sentimento))

             precision    recall  f1-score   support

   Positive       0.80      1.00      0.89    221597
   Negative       0.00      0.00      0.00     26655
    Neutral       0.00      0.00      0.00     30425

avg / total       0.63      0.80      0.70    278677



/home/vagner/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [179]:
# Matriz de confusão

In [180]:
print(pd.crosstab(classifications, resultados, rownames=['True'], colnames=['Predicted'], margins=True))


Predicted  Positive     All
True                       
Negative      26655   26655
Neutral       30425   30425
Positive     221597  221597
All          278677  278677


** Melhorando resultados com Bigrams**

In [181]:
vectorizer = CountVectorizer(ngram_range=(1,2))
freq_reviews = vectorizer.fit_transform(reviews)
modelo = MultinomialNB()
modelo.'fit'(freq_reviews,classifications)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [182]:
resultados = cross_val_predict(modelo, freq_reviews, classifications, cv=10)

In [183]:
metrics.accuracy_score(classifications,resultados)

0.79517505929804033

In [184]:
print (pd.crosstab(classifications, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Positive     All
Real                      
Negative     26655   26655
Neutral      30425   30425
Positive    221597  221597
All         278677  278677


In [185]:
sentimento=['Positive','Negative','Neutral']
print (metrics.classification_report(classifications,resultados,sentimento))

             precision    recall  f1-score   support

   Positive       0.80      1.00      0.89    221597
   Negative       0.00      0.00      0.00     26655
    Neutral       0.00      0.00      0.00     30425

avg / total       0.63      0.80      0.70    278677



/home/vagner/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
